## 0) Plot W_Ekman

- Consider an inflow angle? (either constant or Zhang and Ulhorn? Constant will be enough if I apply it to large number of data)

- Consider a smooth variation of Cd?

In [1]:
# General
import os
import glob

# Display
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.gridspec import GridSpec
import matplotlib.cm as cm

# Data treatment
import xarray as xr
import numpy as np
# from scipy.interpolate import griddata
import pandas as pd 
from tqdm.notebook import tqdm
import ephem

# Default parameters
fontsize = 16
mpl.rcParams.update({'font.size': fontsize})
mpl.rcParams['figure.figsize'] = (15, 10)
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = 'white'

In [2]:
### Paths & Params
PTHS = {
    'sar_pth': '/home/arthur/results/sar_extended/05_addPRIMED/dsg.nc',
    'sav_dir': '/home/arthur/results/TCsLifeMonitFromObs/PRJCT_VERTICAL_WIND/00_PLOT_W/',
}

### Open data
dsg  = xr.open_dataset(PTHS['sar_pth'])

In [3]:
### FUNCTIONS
def holland_profile(r, lat, B, Vmin, Rmax, Vmax):
    '''We assume that rho is constant and equals 1.15 kg.m-3'''
    fcor   = abs(coriolis(lat))
    rho    = 1.15
    r      = r.astype(float) + 0.001           # To avoid dividing by zero
    r_star = (Rmax / r) ** B
    V      = r * 0.
    V      = Vmin + np.sqrt( ((Vmax - Vmin) ** 2) * r_star * np.exp(1 - r_star) + (r * fcor / 2) ** 2) - (r * fcor / 2)
    return V

def coriolis(lat):
    '''Latitude must be in degrees.'''
    Omega = 7.2921e-5                             # Earth rotation vector
    fcor  = 2 * Omega * np.sin(lat * np.pi / 180) # Coriolis parameter assuming it's constant 
    return fcor

rs = np.linspace(10, 500000, 50000) # define dr accordingly
dr = 10 # meters
def vorticity(r, v):
    '''Relative vorticity'''
    return (1 / (r + 0.000001)) * np.gradient(r * v, dr)

def w_ekman(r, v, fcr, Cd=2.5e-3):
    '''Ekman vertical velocity'''
    rvor = vorticity(r, v)
    num  = Cd * r * (v ** 2)
    den  = rvor + fcr
    return (1 / (r + 0.000001)) * np.gradient(num / den, dr)

def w_ekman_varying_Cd(r, v, fcr):
    '''Ekman vertical velocity'''
    # Parametrize Cd
    Cd = 0.49 + 0.065 * v # cf Foster 2009
    Cd[v < 10.92]  = 1.2  # cf Foster 2009
    Cd[v >= 38.62] = 3    # cf Foster 2009
    Cd = Cd * 1e-3
    # Compute w_Ekman
    rvor = vorticity(r, v)
    num  = Cd * r * (v ** 2)
    den  = rvor + fcr
    return (1 / (r + 0.000001)) * np.gradient(num / den, dr)

In [4]:
# FILTERS
cond = (
    (dsg.center_quality_flag < 2) & 
    (dsg.percent_outside < 50) & 
    (dsg.percent_rmx_inside == 1) &
    (abs(dsg.lat) < 30) & 
    (dsg.rmx < 100000) & 
    (dsg.vmx > 20)
)
df   = dsg.where(cond, drop=True)
print(len(df.time))

529


In [5]:
for t in tqdm(range(len(df.time))):
    d = df.isel(time=t)

    ### FIGURE
    fig = plt.figure(figsize = (18, 10))
    gs  = GridSpec(100, 100)
    
    ### Holland profile
    ax  = fig.add_subplot(gs[:, :])
    V   = holland_profile(rs, float(d.lat), float(d.B_hol), float(d.vmn_hol), float(d.rmx_hol), float(d.vmx_hol))
    ax.plot(rs / 1000, V, label='Holland fitted profile', linewidth=3, c='tab:blue')
    ax.axvline(float(d.rmx_hol) / 2000, label='Rmax_hol / 2', c='tab:red')
    # Remove every ticks and make a bold frame
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(2)
    ax.tick_params(direction='inout', width=0, length=0, bottom='off', top='off', left='off', right='off')
    ax.tick_params(direction='inout', width=2, length=5, bottom=1, top=1, left=1, right=1) #set the tick parameters
    ax.grid(ls=':')
    # Labels
    ax.set_ylabel(r'Total wind speed (m/s)', color='tab:blue')
    ax.tick_params(axis='y', labelcolor='tab:blue')
    ax.set_xlabel(r'Distance from center (km)')
    ax.legend(frameon=False, loc='upper center')
    ax.set_xlim(0, 500)
    
    ### w_Ekman
    ax2 = ax.twinx()  # instantiate a second Axes that shares the same x-axis
    w   = w_ekman(rs, V, abs(float(d.fcor)))
    w2  = w_ekman_varying_Cd(rs, V, abs(float(d.fcor)))
    ax2.plot(rs / 1000, w,  label='w_Ekman', c='tab:green')
    ax2.plot(rs / 1000, w2, label='w_Ekman with varying Cd', c='tab:green', linestyle='dashed')
    ax2.set_ylabel('Vertical wind speed (m/s)', color='tab:green')  # we already handled the x-label with ax1
    ax2.tick_params(axis='y', labelcolor='tab:green')
    ax2.legend(frameon=False, loc='upper right')

    ### Save
    plt.savefig(PTHS['sav_dir'] + os.path.splitext(d.file.item())[0], bbox_inches='tight', format='png');plt.clf()

  0%|          | 0/529 [00:00<?, ?it/s]

/tmp/ipykernel_61084/4281319472.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize = (18, 10))


<Figure size 1800x1000 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>

...

<Figure size 1800x1000 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>

<Figure size 1800x1000 with 0 Axes>

In [101]:
# ### FIGURE
# fig = plt.figure(figsize = (18, 10))
# gs  = GridSpec(100, 100)

# ### Holland profile
# ax  = fig.add_subplot(gs[:, :])
# V   = holland_profile(rs, float(d.lat), float(d.B_hol), float(d.vmn_hol), float(d.rmx_hol), float(d.vmx_hol))
# ax.plot(rs / 1000, V, label='Holland fitted profile', linewidth=3, c='tab:blue')
# ax.axvline(float(d.rmx_hol) / 2000, label='Rmax_hol / 2', c='tab:red')
# # Remove every ticks and make a bold frame
# for axis in ['top', 'bottom', 'left', 'right']:
#     ax.spines[axis].set_linewidth(2)
# ax.tick_params(direction='inout', width=0, length=0, bottom='off', top='off', left='off', right='off')
# ax.tick_params(direction='inout', width=2, length=5, bottom=1, top=1, left=1, right=1) #set the tick parameters
# ax.grid(ls=':')
# # Labels
# ax.set_ylabel(r'Total wind speed (m/s)', color='tab:blue')
# ax.tick_params(axis='y', labelcolor='tab:blue')
# ax.set_xlabel(r'Distance from center (km)')
# ax.legend(frameon=False, loc='upper center')
# ax.set_xlim(0, 500)

# # w_Ekman
# ax2 = ax.twinx()  # instantiate a second Axes that shares the same x-axis
# w   = w_ekman(rs, V, abs(float(d.fcor)))
# w2  = w_ekman_varying_Cd(rs, V, abs(float(d.fcor)))
# ax2.plot(rs / 1000, w,  label='w_Ekman', c='tab:green')
# ax2.plot(rs / 1000, w2, label='w_Ekman with varying Cd', c='tab:green', linestyle='dashed')
# ax2.set_ylabel('Vertical wind speed (m/s)', color='tab:green')  # we already handled the x-label with ax1
# ax2.tick_params(axis='y', labelcolor='tab:green')
# ax2.legend(frameon=False, loc='upper right')

In [92]:
# ### FIGURE
# fig = plt.figure(figsize = (12, 15))
# gs  = GridSpec(100, 100)

# ### Holland profile
# ax  = fig.add_subplot(gs[:46, :])
# V   = holland_profile(rs, float(d.lat), float(d.B_hol), float(d.vmn_hol), float(d.rmx_hol), float(d.vmx_hol),)
# ax.plot(rs / 1000, V, label='Holland fitted profile')
# ax.axvline(float(d.rmx_hol) / 2000, label='Rmax_hol / 2', c='tab:red')
# # Remove every ticks and make a bold frame
# for axis in ['top', 'bottom', 'left', 'right']:
#     ax.spines[axis].set_linewidth(2)
# ax.tick_params(direction='inout', width=0, length=0, bottom='off', top='off', left='off', right='off')
# ax.tick_params(direction='inout', width=2, length=5, bottom=1, top=1, left=1, right=1) #set the tick parameters
# ax.grid(ls=':')
# # Labels
# ax.set_ylabel(r'Total wind speed (m/s)')
# ax.set_xlabel(r'Distance from center (km)')
# ax.legend(frameon=False, loc='upper right')
# ax.set_xlim(0, 500)

# ### w_Ekman
# ax  = fig.add_subplot(gs[54:, :])
# w   = w_ekman(rs, V, abs(float(d.fcor)))
# ax.plot(rs / 1000, w, label='w_Ekman')
# ax.axvline(float(d.rmx_hol) / 2000, label='Rmax_hol / 2', c='tab:red')
# # Remove every ticks and make a bold frame
# for axis in ['top', 'bottom', 'left', 'right']:
#     ax.spines[axis].set_linewidth(2)
# ax.tick_params(direction='inout', width=0, length=0, bottom='off', top='off', left='off', right='off')
# ax.tick_params(direction='inout', width=2, length=5, bottom=1, top=1, left=1, right=1) #set the tick parameters
# ax.grid(ls=':')
# # Labels
# ax.set_ylabel(r'Total wind speed (m/s)')
# ax.set_xlabel(r'Distance from center (km)')
# ax.legend(frameon=False, loc='upper right')
# ax.set_xlim(0, 500)